In [1]:
# Crivo - pedidos
# Pego alguma base de performance e relaciono pedido com codigo do contrato.
# Eu pego o ultimo pedido - seria interessante reforcar isso com o Iverson.
# NECESSARIO - Comparar o crivo que eu tenho com o novo que foi gerado para verificar
#              diferenca nas datas e nao ter repeticao.
#              ADICIONALMENTE - usar codigo proposta como primary key

# dar um join pedido - contrato e ficar com o que sobrar


In [2]:
import pandas as pd
import dwsemear
import datetime
import collections
import numpy as np

import postgresemear as sql  # mover os arquivos para a pasta principal

def calcula_idade(dados, coluna):
    idade = []
    for i in dados.index:
        if dwsemear.e_nulo(dados.loc[i,coluna]):
            idade.append(None)
        else:
            anos = (dados.loc[i,'datahora'] - dados.loc[i,coluna]).days
            anos = int(round(anos/365))
            idade.append(anos)
    dados['idade'] = idade
    return dados

def remove_localize(data):
    return data.tz_localize(None)

def coluna_para_lista(coluna):
    nova_coluna = []
    for elem in coluna:
        if dwsemear.e_nulo(elem):
            nova_coluna.append(None)
        else:
            nova_coluna.append(elem)
    return nova_coluna

def para_int(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return int(elemento)

def para_float(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return float(elemento)

def para_str(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return str(elemento)
    
def para_data(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return elemento
    
def pegue_elementos_com_repeticao(coluna):
    contagem = collections.Counter(coluna)
    return list(getItemsUpToCount(contagem,1).keys())

def getItemsUpToCount(dct, n):
    upToCount = {}
    for key in dct:
        if dct[key] > n:
            upToCount[key] = dct[key]
    return upToCount


In [3]:
crivo = pd.read_csv('DadosCrivo-NOV2018.csv',sep=';', encoding='latin-1',dtype=str)

In [4]:
crivo = dwsemear.limpa_nome_das_colunas(crivo)
crivo = dwsemear.coluna_cpf_para_inteiro(crivo, 'CPF')

In [5]:
# Obtendo timestamp do evento (data + hora)
datahora = []
for i in crivo.index:
    datahora.append(crivo.loc[i,' Data'] + ' ' + crivo.loc[i,' Hora'])
crivo['datahora'] = datahora    
crivo['datahora'] = pd.to_datetime(crivo['datahora'], format='%d/%m/%Y %I:%M:%S %p')

In [6]:
# Obtendo proposta unica (a ultima realizada)
crivo.loc[:,'Codigo proposta'] = crivo.loc[:,'Codigo proposta'].astype(int)
indice_mais_recente = []
for proposta in set(crivo.loc[:,'Codigo proposta']):
    crivo_proposta = crivo[crivo.loc[:,'Codigo proposta'] == proposta]
    ultimo_pedido = crivo_proposta[crivo_proposta.loc[:,'Codigo proposta'] == proposta].datahora.max()
    indice = crivo_proposta[(crivo_proposta.loc[:,'Codigo proposta'] == proposta) &
      (crivo_proposta.loc[:,'datahora'] == ultimo_pedido)].index[0]
    indice_mais_recente.append(indice)
indice_mais_recente.sort()
coluna_mais_recente = np.zeros(len(crivo),dtype=int)
for i in indice_mais_recente:
    coluna_mais_recente[i] = 1
crivo['ultimo_pedido'] = coluna_mais_recente    

In [7]:
coluna = 'Data Nascimento Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = pd.to_datetime(crivo[coluna], format='%d/%m/%Y')
crivo = calcula_idade(crivo, coluna) # idade

In [8]:
#coluna = 'Valor Compra Banco Semear Consulta Proposta JRetail PF '
#crivo[coluna] = crivo[coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Renda Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = crivo[coluna].apply(dwsemear.converte_latin_para_float)

In [9]:
# 'Discreta Sexo Banco Semear Consulta Proposta JRetail PF '
# crivo.loc[:,'Discreta UF Banco Semear Consulta Proposta JRetail PF ']
# lembrar que data virou datahora
# Discreta Estado civil Banco Semear Consulta Proposta JRetail PF 

In [10]:
coluna = 'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ' # esse e o score?
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [11]:
coluna = 'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [12]:
coluna = 'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [13]:
coluna = 'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [14]:
coluna = 'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [15]:
coluna = 'Valor Renda presumida Unitfour Consulta perfil PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [16]:
coluna = 'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [17]:
id_crivo = 0
crivo_registros = []
for i in crivo.index:
    id_crivo += 1
#    if id_crivo > 10:
#        break

    crivo_registros.append((id_crivo,
          para_int(crivo.loc[i,'Codigo proposta']),
          para_data(crivo.loc[i,'datahora']),  # esse cara poderia ser not null
          para_str(crivo.loc[i,'Discreta UF Banco Semear Consulta Proposta JRetail PF ']),
          para_str(crivo.loc[i,'Discreta Sexo Banco Semear Consulta Proposta JRetail PF ']),
          para_float(crivo.loc[i,'Valor Renda Banco Semear Consulta Proposta JRetail PF ']),
          para_int(crivo.loc[i,'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ']),
          para_float(crivo.loc[i,'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF ']),
          para_float(crivo.loc[i,'Valor Renda presumida Unitfour Consulta perfil PF ']),
          para_float(crivo.loc[i,'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF ']),
          para_int(crivo.loc[i,'ultimo_pedido']),
          para_int(crivo.loc[i,'idade']),  # not null
          para_int(crivo.loc[i,'CPF']),
          para_int(crivo.loc[i,'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF ']),
          para_int(crivo.loc[i,'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF ']),
          para_int(crivo.loc[i,'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '])))
        
#    print('LINHA: ',
#          para_int(crivo.loc[i,'Codigo proposta']),
#          para_data(crivo.loc[i,'datahora']),  # esse cara poderia ser not null
#          para_str(crivo.loc[i,'Discreta UF Banco Semear Consulta Proposta JRetail PF ']),
#          para_str(crivo.loc[i,'Discreta Sexo Banco Semear Consulta Proposta JRetail PF ']),
#          para_float(crivo.loc[i,'Valor Renda Banco Semear Consulta Proposta JRetail PF ']),
#          para_float(crivo.loc[i,'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ']),
#          para_float(crivo.loc[i,'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF ']),
#          para_float(crivo.loc[i,'Valor Renda presumida Unitfour Consulta perfil PF ']),
#          para_float(crivo.loc[i,'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF ']),
#          para_int(crivo.loc[i,'idade']),  # not null
#          para_int(crivo.loc[i,'CPF']),
#          para_int(crivo.loc[i,'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF ']),
#          para_int(crivo.loc[i,'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF ']),
#          para_int(crivo.loc[i,'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '])
#         )
    

# LOAD

In [26]:
delete_tabela = '''DROP TABLE crivo'''
sql.execute_query(delete_tabela)

In [27]:
crie_tabela = '''CREATE TABLE crivo
            (ID BIGINT PRIMARY KEY NOT NULL,
            codigo_proposta BIGINT NOT NULL,
            data timestamp with time zone,
            uf TEXT,
            genero TEXT,
            renda_cliente REAL,
            score INT,
            valor_parcela REAL,
            renda_presumida_unitfour REAL,
            atraso_medio REAL,
            ultimo_pedido INT,
            idade INT,
            cpf BIGINT,
            quantidade_parcelas INT,
            associados_distintos_30_dias INT,
            quantidade_consultas INT);'''
sql.execute_query(crie_tabela)

In [28]:
query = """ INSERT INTO crivo
            (ID,
            codigo_proposta,
            data,
            uf,
            genero,
            renda_cliente,
            score,
            valor_parcela,
            renda_presumida_unitfour,
            atraso_medio,
            ultimo_pedido,
            idade,
            cpf,
            quantidade_parcelas,
            associados_distintos_30_dias,
            quantidade_consultas) 
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
sql.adicione_multiplos_registros(query, crivo_registros)

# TESTANDO

In [21]:
parametros = [1,31]
dados_crivo = sql.query_para_pandas('''SELECT *
                      FROM crivo
                      WHERE ID BETWEEN %s AND %s
                      ''',parametros)

In [ ]:
#coluna = 'Data Admiss o Banco Semear Consulta Proposta JRetail PF ' - corrigir isso aqui
#crivo[coluna] = pd.to_datetime(crivo[coluna], format='%d/%m/%Y')
#crivo = calcula_idade(crivo, coluna) # idade
#tempo que mora na mesma casa

In [ ]:
for col in crivo.columns:
    if 'traso' in col:
        print(col)

# CRIVO ANTERIOR

In [ ]:
import pandas as pd
import dwsemear
import datetime
import collections

import postgresemear as sql  # mover os arquivos para a pasta principal

def calcula_idade(dados):
    idade = []
    nasc = 'Data Nascimento Banco Semear Consulta Proposta JRetail PF '
    for i in dados.index:
        anos = (dados.loc[i,' Data'] - dados.loc[i,nasc]).days
        anos = int(round(anos/365))
        idade.append(anos)
    dados['idade'] = idade
    return dados

def remove_localize(data):
    return data.tz_localize(None)

In [ ]:
file = 'CreditDefaultSemear-v2.csv'
dados = pd.read_csv(file, sep=';', encoding='latin-1', dtype=str)

In [ ]:
dados = dwsemear.limpa_nome_das_colunas(dados)
dados = dwsemear.coluna_cpf_para_inteiro(dados, 'CPF x')
dados[' Data'] = pd.to_datetime(dados[' Data'], format='%d/%m/%Y')
# UF y
coluna = 'Data Nascimento Banco Semear Consulta Proposta JRetail PF '
dados[coluna] = pd.to_datetime(dados[coluna], format='%d/%m/%Y')
dados = calcula_idade(dados) # idade
coluna = 'Valor Compra Banco Semear Consulta Proposta JRetail PF '
dados[coluna] = dados[coluna].astype(float)
coluna = 'Valor Renda Banco Semear Consulta Proposta JRetail PF '
dados[coluna] = dados[coluna].apply(dwsemear.converte_latin_para_float)
# 'Discreta Sexo Banco Semear Consulta Proposta JRetail PF '
# 'Contrato Jretail'
# 'Con Contrato'

In [ ]:
k = 0
crivo_registros = []
for i in dados.index:
    k += 1
#    if k > 10:
#        break

    crivo_registros.append((
        k,
        dados.loc[i, ' Data'],
        int(dados.loc[i,'Con Contrato']),
        int(dados.loc[i,'idade']),
        dados.loc[i,'UF y'],
        dados.loc[i,'Discreta Sexo Banco Semear Consulta Proposta JRetail PF '],
        dados.loc[i,'Valor Renda Banco Semear Consulta Proposta JRetail PF '],
        dados.loc[i,'Valor Compra Banco Semear Consulta Proposta JRetail PF '],
        int(dados.loc[i,'Inadimplente'])))
        
        
#    print('LINHA: ',
#          int(dados.loc[i,'Con Contrato']),
#          dados.loc[i,'idade'],
#          dados.loc[i,'UF y'],
#          dados.loc[i,'Discreta Sexo Banco Semear Consulta Proposta JRetail PF '],
#          dados.loc[i,'Valor Renda Banco Semear Consulta Proposta JRetail PF '],
#          dados.loc[i,'Valor Compra Banco Semear Consulta Proposta JRetail PF '],
#          dados.loc[i,'Inadimplente'] # 60 dias
#         )
    

In [ ]:
delete_tabela = '''DROP TABLE crivo'''
sql.execute_query(delete_tabela)

In [ ]:
crie_tabela = '''CREATE TABLE crivo
            (ID BIGINT PRIMARY KEY NOT NULL,
            Data timestamp with time zone NOT NULL,
            Contrato BIGINT NOT NULL,
            Idade REAL NOT NULL,
            Uf TEXT NOT NULL,
            Genero TEXT NOT NULL,
            Valor_renda REAL NOT NULL,
            Valor_compra REAL NOT NULL,
            Inadimplente INT NOT NULL);'''
sql.execute_query(crie_tabela)

In [ ]:
query = """ INSERT INTO crivo
            (ID,
            Data,
            Contrato,
            Idade,
            Uf,
            Genero,
            Valor_renda,
            Valor_compra,
            Inadimplente) 
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
sql.adicione_multiplos_registros(query, crivo_registros)

In [ ]:
COLUMNS = ['ID', 'Data', 'Contrato',
           'Idade', 'Uf', 'Genero',
           'Valor_renda', 'Valor_compra',
           'Inadimplente']

In [ ]:
parametros = [1,31]
dados_crivo = sql.query_para_pandas('''SELECT *
                      FROM crivo
                      WHERE ID BETWEEN %s AND %s
                      ''',parametros)
dados_crivo.columns = COLUMNS

In [ ]:
dados_crivo.Data = dados_crivo.Data.apply(remove_localize)